In [1]:
!which python

/lfs/hyperturing1/0/kaif/miniforge/envs/sc/bin/python


In [1]:
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "4,6,7,8"

from vllm import LLM, SamplingParams
from dataset import load_datasets, BOXED_ANSWERS_DATASETS, get_boxed_answer
from latex_formater import latex_deformat
from prompt import generate_nshot_prompts

In [5]:
??SamplingParams

In [2]:
sampling_params = SamplingParams(temperature=0.8, top_p=0.95, max_tokens=1028)
model_name = "meta-llama/Llama-2-7b-hf"
llm = LLM(model=model_name, trust_remote_code=True, tensor_parallel_size=4)

INFO 02-27 14:02:56 config.py:413] Custom all-reduce kernels are temporarily disabled due to stability issues. We will re-enable them once the issues are resolved.


2024-02-27 14:02:58,955	INFO worker.py:1724 -- Started a local Ray instance.


INFO 02-27 14:03:00 llm_engine.py:79] Initializing an LLM engine with config: model='meta-llama/Llama-2-7b-hf', tokenizer='meta-llama/Llama-2-7b-hf', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=auto, tensor_parallel_size=4, disable_custom_all_reduce=True, quantization=None, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, seed=0)
(RayWorkerVllm pid=1272414) INFO 02-27 14:03:14 weight_utils.py:163] Using model weights format ['*.safetensors']
INFO 02-27 14:03:14 weight_utils.py:163] Using model weights format ['*.safetensors']
INFO 02-27 14:03:18 llm_engine.py:337] # GPU blocks: 18453, # CPU blocks: 2048
INFO 02-27 14:03:20 model_runner.py:676] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 02-27 14:03:20 mode

In [3]:
%%capture
datasets = load_datasets(BOXED_ANSWERS_DATASETS);

A board game spinner is divided into three parts labeled $A$, $B$  and $C$.
The probability of the spinner landing on $A$ is $\\frac{1}{3}$ and the
probability of the spinner landing on $B$ is $\\frac{5}{12}$.  What is
the probability of the spinner landing on $C$? Express your answer as a
common fraction.

Solution: The spinner is guaranteed to land on exactly one of the three
regions, so we know that the sum of the probabilities of it landing in
each region will be 1. If we let the probability of it landing in region
$C$ be $x$, we then have the equation $1 = \\frac{5}{12}+\\frac{1}{3}+x$,
from which we have $x=\\boxed{\\frac{1}{4}}$.

Above is a question and correct solution. Propose a place in the solution
where a normal student might make a reasoning mistake. Then explain the
error beginning with "Explanation:".
~~
The student might make a mistake when summing the probabilities, and more
specifically when making the conversion between fractions and whole numbers.

Explanation: A common error might involve not correctly accounting for the
total probability being 1. For instance, the student might disregard the
conversion between different types of fractions. In this case, the student
may add the fractions incorrectly, perhaps adding $\frac{1}{3}$ and
$\frac{5}{12}$ as if they are like fractions, resulting in a incorrect sum
of $\frac{6}{15}$ instead of the correct sum $\frac{13}{12}$, and
subsequently derive an incorrect value for $x$. 
~~
Now construct the students complete response that makes this mistake. Do
NOT point out the mistake in the response. Adhere to the original
formatting. Begin with "Solution:".
~~
Solution: The spinner is guaranteed to land on exactly one of the three
regions, so we know that the sum of the probabilities of it landing in
each region will be 1. If we let the probability of it landing in region
$C$ be $x$, we then have the equation $1 = \frac{5}{12}+\frac{1}{3}+x$. 

When adding the fractions $\frac{5}{12}$ and $\frac{1}{3}$ directly, we
get $\frac{6}{15}$. Subtracting this from 1 gives us $1 - \frac{6}{15}
= \frac{9}{15}$, which simplifies to $x = \boxed{\frac{3}{5}}$.

In [8]:
DATASET_MAP = {
    'EleutherAI/hendrycks_math_algebra': 'Algebra',
    'EleutherAI/hendrycks_math_counting_and_probability': 'Counting and Probability',
    'EleutherAI/hendrycks_math_geometry': 'Geometry',
    'EleutherAI/hendrycks_math_intermediate_algebra': 'Intermediate Algebra',
    'EleutherAI/hendrycks_math_number_theory': 'Number Theory',
    'EleutherAI/hendrycks_math_prealgebra': 'Prealgebra',
    'EleutherAI/hendrycks_math_precalculus': 'Precalculus',
    'gsm8k_main': 'GSM8K'
}

MODEL_MAP = {
    'deepseek-ai/deepseek-math-7b-rl': 'Deepseek-7b-RL',
    'deepseek-ai/deepseek-math-7b-instruct': 'Deepseek-7b-Instruct',
    'llm-agents/tora-13b-v1.0': 'Tora-13b-v1.0',
    'EleutherAI/llemma_7b': 'LLeMMA-7b',
    'llm-agents/tora-7b-v1.0': 'Tora-7b-v1.0',
    'google/gemma-7b': 'Gemma-7b',
    'morph-labs/morph-prover-v0-7b': 'Morph-7b-v0',
    'lmsys/vicuna-13b-v1.5': 'Vicuna-13b-v1.5',
    'lmsys/vicuna-7b-v1.5': 'Vicuna-7b-v1.5',
    'mistralai/Mistral-7B-Instruct-v0.2': 'Mistral-7b-Instruct-v0.2',
    'meta-llama/Llama-2-13b-chat-hf': 'Llama-2-13b',
    'meta-llama/Llama-2-7b-hf': 'Llama-2-7b'
}

In [5]:
datasets = {DATASET_MAP[dataset['name']]: generate_nshot_prompts(dataset['data']['train'], n=3) for dataset in datasets}

In [6]:
# print(prompts['Algebra'][1]['question'])
# prompt the model for the answers for the question
from util import save_output


@save_output
def generate(dataset, model, sampling_params):
    prompts = [p['question'] for p in dataset]
    outputs = model.generate(prompts, sampling_params)
    return outputs

In [ ]:
from pathlib import Path

save_dir = Path("~/generated_predictions").expanduser()
save_dir.mkdir(exist_ok=True)
load = True
predictions = {
    dataset: generate(
        prompts, llm, sampling_params, load=load,
        output_path=save_dir / f"{dataset}_{MODEL_MAP[model_name]}_predictions.pkl"
    ) for dataset, prompts in datasets.items()
}

Processed prompts:   0%|                                                                 | 0/1741 [00:00<?, ?it/s]

In [15]:
from dataset import get_boxed_answer
from latex_formater import latex_deformat


def grade(answers: list[str], targets: list[str]) -> list:
    return [a == t for a, t in zip(answers, targets)]


def grade_predictions(outputs, data):
    boxed_predictions = []
    for o in outputs:
        try:
            boxed_predictions.append(latex_deformat(get_boxed_answer(o.outputs[0].text)))
        except:
            print(get_boxed_answer(o.outputs[0].text))

    # boxed_predictions = [latex_deformat(get_boxed_answer(o.outputs[0].text)) for o in outputs]
    boxed_answers = [latex_deformat(get_boxed_answer(d)) for d in data]
    grades = grade(boxed_predictions, boxed_answers)

    return grades

In [30]:
print(predictions['Algebra'][0].prompt)
print(predictions['Algebra'][0].outputs[0].text)
print(datasets['Algebra'][0]['answer'])

Answer the following 4 questions:

1. Let \[f(x) = \left\{
\begin{array}{cl} ax+3, &\text{ if }x>2, \\
x-5 &\text{ if } -2 \le x \le 2, \\
2x-b &\text{ if } x <-2.
\end{array}
\right.\]Find $a+b$ if the piecewise function is continuous (which means that its graph can be drawn without lifting your pencil from the paper).

For the piecewise function to be continuous, the cases must "meet" at $2$ and $-2$. For example, $ax+3$ and $x-5$ must be equal when $x=2$. This implies $a(2)+3=2-5$, which we solve to get $2a=-6 \Rightarrow a=-3$. Similarly, $x-5$ and $2x-b$ must be equal when $x=-2$. Substituting, we get $-2-5=2(-2)-b$, which implies $b=3$. So $a+b=-3+3=\boxed{0}$.


2. A rectangular band formation is a formation with $m$ band members in each of $r$ rows, where $m$ and $r$ are integers. A particular band has less than 100 band members. The director arranges them in a rectangular formation and finds that he has two members left over. If he increases the number of members in each row b

In [22]:
grades = {dataset: grade_predictions(predictions[dataset], [p['answer'] for p in data]) for dataset, data in
          datasets.items()}

In [25]:
import numpy as np
{dataset: np.mean(grade) for dataset, grade in grades.items()}

{'Algebra': 0.026421596783457783,
 'Counting and Probability': 0.01171875,
 'Geometry': 0.0034602076124567475,
 'Intermediate Algebra': 0.0030959752321981426,
 'Number Theory': 0.023094688221709007,
 'Prealgebra': 0.04409317803660566,
 'Precalculus': 0.0,
 'GSM8K': 0.9925048517700595}

In [14]:
# save the predictions
# for dataset, outputs in predictions.items():
#     with open(save_dir / f"{dataset.replace(' ', '-')}_{MODEL_MAP[model_name]}_predictions.pkl", "wb") as f:
#         pickle.dump(outputs, f)

In [5]:
def generate(prompts, model, sampling_params, chat_mode=False):
    outputs = llm.generate(prompts, sampling_params)
    if chat_mode:
        return [prompt + o.outputs[0].text for prompt, o in zip(prompts, outputs)]
    else:
        return [o.outputs[0].text for o in outputs]

In [7]:
prompts = ["""A board game spinner is divided into three parts labeled $A$, $B$  and $C$.
The probability of the spinner landing on $A$ is $\\frac{1}{3}$ and the
probability of the spinner landing on $B$ is $\\frac{5}{12}$.  What is
the probability of the spinner landing on $C$? Express your answer as a
common fraction. Write your final answer as "\\boxed{...}".

Solution:
"""]
# Solution: The spinner is guaranteed to land on exactly one of the three
# regions, so we know that the sum of the probabilities of it landing in
# each region will be 1. If we let the probability of it landing in region
# $C$ be $x$, we then have the equation $1 = \\frac{5}{12}+\\frac{1}{3}+x$,
# from which we have $x=\\boxed{\\frac{1}{4}}$.

# Above is a question and correct solution. What is a mistake someone might make when solving the problem?

# Mistake:"""]
outputs = generate(prompts, llm, sampling_params, chat_mode=True)
print(outputs[0])

Processed prompts: 100%|████████████████████████████████████████████████████████████| 1/1 [00:24<00:00, 24.34s/it]

A board game spinner is divided into three parts labeled $A$, $B$  and $C$.
The probability of the spinner landing on $A$ is $\frac{1}{3}$ and the
probability of the spinner landing on $B$ is $\frac{5}{12}$.  What is
the probability of the spinner landing on $C$? Express your answer as a
common fraction. Write your final answer as "\boxed{...}".

Solution:
We want to find the probability of the spinner landing on $C$, which means that it must land on $A$ or $B$ and not on $C$. We can find this by using the formula for conditional probability:

P(C) = P(C|A) \* P(A) + P(C|B) \* P(B)

We know that P(A) = $\frac{1}{3}$ and P(B) = $\frac{5}{12}$, so we need to find P(C|A) and P(C|B).

P(C|A) means the probability of the spinner landing on $C$ given that it landed on $A$. To find this, we need to know the probability of the spinner landing on $B$ or $C$ given that it landed on $A$. However, we don't have that information, so we will assume that the probabilities are equal. That is, P(B|A) =

In [14]:
prompts2 = [o + """

Are you sure this is correct?
""" for o in outputs]
outputs2 = generate(prompts2, llm, sampling_params, chat_mode=True)
print(outputs2[0])

Processed prompts: 100%|████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.66it/s]

A board game spinner is divided into three parts labeled $A$, $B$  and $C$.
The probability of the spinner landing on $A$ is $\frac{1}{3}$ and the
probability of the spinner landing on $B$ is $\frac{5}{12}$.  What is
the probability of the spinner landing on $C$? Express your answer as a
common fraction. Write your final answer as "\boxed{...}".

Solution:
We want to find the probability of the spinner landing on $C$, which means that it must land on $A$ or $B$ and not on $C$. We can find this by using the formula for conditional probability:

P(C) = P(C|A) \* P(A) + P(C|B) \* P(B)

We know that P(A) = $\frac{1}{3}$ and P(B) = $\frac{5}{12}$, so we need to find P(C|A) and P(C|B).

P(C|A) means the probability of the spinner landing on $C$ given that it landed on $A$. To find this, we need to know the probability of the spinner landing on $B$ or $C$ given that it landed on $A$. However, we don't have that information, so we will assume that the probabilities are equal. That is, P(B|A) =

In [27]:
prompts2 = [o + """

Now construct a response that makes this mistake followed 
by a teacher explaining the mistake made. Begin with "Incorrect Solution:" 
and the teacher corrections with "Teacher Correction:".

Incorrect Solution: """ for o in outputs]
outputs2 = generate(prompts2, llm, sampling_params, chat_mode=True)
print(outputs2[0])

Processed prompts: 100%|████████████████████████████████████████████████████████████| 1/1 [00:22<00:00, 22.87s/it]

A board game spinner is divided into three parts labeled $A$, $B$  and $C$.
The probability of the spinner landing on $A$ is $\frac{1}{3}$ and the
probability of the spinner landing on $B$ is $\frac{5}{12}$.  What is
the probability of the spinner landing on $C$? Express your answer as a
common fraction.

Solution: The spinner is guaranteed to land on exactly one of the three
regions, so we know that the sum of the probabilities of it landing in
each region will be 1. If we let the probability of it landing in region
$C$ be $x$, we then have the equation $1 = \frac{5}{12}+\frac{1}{3}+x$,
from which we have $x=\boxed{\frac{1}{4}}$.

Above is a question and correct solution. What is a mistake someone might make when solving the problem?

Mistake: One possible mistake is assuming that the probabilities of the spinner landing on each of the three regions must be equal, and therefore setting $x=\frac{1}{3}$. This is incorrect, as the given probabilities for $A$ and $B$ already show that the